<a href="https://colab.research.google.com/github/Faiz-Nadeem/AI-Skincare-Safety-Checker/blob/main/AI_Skincare_Safety_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
files.upload()  # Upload kaggle.json here when prompted


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"faiznadeem","key":"1cd63eb41570987df334336fa212f064"}'}

In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [8]:
!kaggle datasets download -d eward96/skincare-products-clean-dataset
!unzip skincare-products-clean-dataset.zip


Dataset URL: https://www.kaggle.com/datasets/eward96/skincare-products-clean-dataset
License(s): unknown
  0% 0.00/152k [00:00<?, ?B/s]
100% 152k/152k [00:00<00:00, 459MB/s]
Archive:  skincare-products-clean-dataset.zip
  inflating: skincare_products_clean.csv  


In [10]:
import pandas as pd

df = pd.read_csv('skincare_products_clean.csv')
df.head()


,product_name,product_url,product_type,clean_ingreds,price
0,The Ordinary Natural Moisturising Factors + HA...,https://www.lookfantastic.com/the-ordinary-nat...,Moisturiser,"['capric triglyceride', 'cetyl alcohol', 'prop...",£5.20
1,CeraVe Facial Moisturising Lotion SPF 25 52ml,https://www.lookfantastic.com/cerave-facial-mo...,Moisturiser,"['homosalate', 'glycerin', 'octocrylene', 'eth...",£13.00
2,The Ordinary Hyaluronic Acid 2% + B5 Hydration...,https://www.lookfantastic.com/the-ordinary-hya...,Moisturiser,"['sodium hyaluronate', 'sodium hyaluronate', '...",£6.20
3,AMELIORATE Transforming Body Lotion 200ml,https://www.lookfantastic.com/ameliorate-trans...,Moisturiser,"['ammonium lactate', 'c12-15', 'glycerin', 'pr...",£22.50
4,CeraVe Moisturising Cream 454g,https://www.lookfantastic.com/cerave-moisturis...,Moisturiser,"['glycerin', 'cetearyl alcohol', 'capric trigl...",£16.00


In [12]:
print(df.columns)


Index(['product_name', 'product_url', 'product_type', 'clean_ingreds',
       'price'],
      dtype='object')


In [17]:

!pip install easyocr
!pip install gradio
!pip install opencv-python-headless


import easyocr
import numpy as np
import gradio as gr
from PIL import Image

reader = easyocr.Reader(['en'], gpu=False)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [18]:
sensitive_skin_avoid = [
    'alcohol', 'fragrance', 'parfum', 'essential oil', 'menthol',
    'eucalyptus', 'peppermint', 'cinnamon', 'sodium lauryl sulfate',
    'formaldehyde', 'camphor', 'citral', 'geraniol', 'limonene',
    'linalool', 'methylisothiazolinone'
]
oily_skin_avoid = [
    'coconut oil', 'lanolin', 'mineral oil', 'petrolatum',
    'isopropyl myristate', 'isopropyl palmitate', 'myristyl myristate',
    'lauric acid', 'butyl stearate', 'oleic acid'
]
dry_skin_avoid = [
    'salicylic acid', 'benzoyl peroxide', 'retinoic acid',
    'retinol', 'witch hazel', 'tea tree oil', 'alpha-hydroxy acids (aha)'
]


In [28]:
def is_safe_for_skin_type(text, skin_type):
    text = text.lower()

    bad = []
    good = []
    tips = ""
    suggestions = []

    if skin_type == "sensitive":
        bad = [i for i in sensitive_skin_avoid if i in text]
        good = [i for i in ['niacinamide', 'ceramide'] if i in text]
        tips = "Look for fragrance-free, alcohol-free products. Niacinamide and ceramides calm irritation."
        suggestions = ["CeraVe Moisturizing Cream", "La Roche-Posay Toleriane", "Vanicream Gentle Cleanser"]

    elif skin_type == "oily":
        bad = [i for i in oily_skin_avoid if i in text]
        good = [i for i in ['salicylic acid', 'niacinamide'] if i in text]
        tips = "Go for oil-free, non-comedogenic products. Salicylic acid and niacinamide reduce oil."
        suggestions = ["Neutrogena Oil-Free Acne Wash", "Paula's Choice BHA", "The Ordinary Niacinamide"]

    elif skin_type == "dry":
        bad = [i for i in dry_skin_avoid if i in text]
        good = [i for i in ['hyaluronic acid', 'glycerin', 'ceramide'] if i in text]
        tips = "Look for hydrating ingredients like hyaluronic acid and ceramides. Avoid harsh exfoliants."
        suggestions = ["CeraVe Hydrating Cleanser", "Aveeno Daily Moisturizing", "Eucerin Advanced Repair"]

    if bad:
        return f"""
Not recommended for {skin_type} skin.
Ingredients to avoid: {', '.join(bad)}

Tip: {tips}
"""
    else:
        good_line = f"\n Beneficial ingredients found: {', '.join(good)}" if good else ""
        return f"""
Safe for {skin_type} skin!{good_line}

Tip: {tips}

Recommended products:
- {suggestions[0]}
- {suggestions[1]}
- {suggestions[2]}
"""


In [29]:
def process_image(image, skin_type):
    img = np.array(image)
    result = reader.readtext(img, detail=0)
    extracted_text = " ".join(result)

    if not extracted_text:
        return " No text detected. Please upload a clearer image."

    safety_result = is_safe_for_skin_type(extracted_text, skin_type)
    return f" **Extracted Ingredients/Text:**\n{extracted_text}\n\n---\n\n{str(safety_result)}"


In [30]:
gr.Interface(
    fn=process_image,
    inputs=[
        gr.Image(type="pil", label="📸 Upload Product Image"),
        gr.Dropdown(["sensitive", "oily", "dry"], label="💆 Select Your Skin Type")
    ],
    outputs="text",
    title="AI Skincare Safety Checker",
    description="Upload a product image to check if it's safe for your skin type. 🔍"
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://94d381ae63c0e5c3f0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
